In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/janelgbmstackedlsvc-1/lgbs.bin
/kaggle/input/janelgbmstackedlsvc-1/sclfs.bin
/kaggle/input/janelgbmstackedlsvc-1/submission.csv
/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py
/kaggle/input/jane-stacked-lgbm-rishi/lgbm_1.bin
/kaggle/input/jane-stacked-lgbm-rishi/lgbm_2.bin
/kaggle/input/jane-stacked-lgbm-rishi/sclf.bin


In [2]:
import gc
import random
from   tqdm import tqdm
from   sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import psutil
import datatable as dt
from   collections import namedtuple
from   sklearn.pipeline import Pipeline
from   sklearn.impute import SimpleImputer
from   sklearn.preprocessing import StandardScaler
from   sklearn.pipeline import Pipeline
import lightgbm as lgb
from   sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pickle
from   mlxtend.classifier import StackingClassifier, StackingCVClassifier
from   sklearn.calibration import CalibratedClassifierCV
import os
import seaborn as sns
from   sklearn.utils import shuffle
import datetime
import time
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import balanced_accuracy_score, recall_score, precision_score, confusion_matrix, make_scorer, f1_score, accuracy_score, precision_recall_fscore_support, matthews_corrcoef, roc_auc_score

from scipy.sparse import csr_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb
from sklearn.svm import LinearSVC
from copy import deepcopy

import torch

import warnings
warnings.filterwarnings ("ignore")
VERSION = ''

In [3]:
# Global Params
SEED         = 420
N_ESTIMATORS = 550
DEVICE       = torch.device ("cpu")

# Helpers

In [4]:
CV_SCORERS = {
    'precision_score':         make_scorer (precision_score),
    'recall_score':            make_scorer (recall_score),
    'f1_score':                make_scorer (f1_score),
    'balanced_accuracy_score': make_scorer (balanced_accuracy_score)
}

In [5]:
def pickleSave (obj, file):

    if VERSION != '':

        file, ext = os.path.splitext (file)
        file += "_v" + str (VERSION) + ext
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):

    if VERSION != '':

        file, ext = os.path.splitext(file)
        file += "_v" + str(VERSION) + ext
    return pickle.load (open (file, 'rb'))

In [6]:
class Best_clf_cv_transformer (BaseEstimator, TransformerMixin): 
    
    def __init__(self, myparams={'name':'LSvc', 'C':1}, **other_params):
        
        self.myparams = myparams
        self.myinit (**other_params)
        return
    
    def myinit (self, **other_params):
        
        self.cv    =  5
        if 'cv' in self.myparams:
            self.cv= self.myparams['cv']
        clf        =  None
        name       =  self.myparams['name']
        if name   == 'Logit':
            clf    =  LogisticRegression (random_state=0)
        elif name == 'DT':
            clf    =  DecisionTreeClassifier (random_state=0)
        elif name == 'RidgClf':
            clf    =  RidgeClassifier (random_state=0)
        elif name == 'Prcpt':
            clf    =  Perceptron (random_state=0)
        elif name == 'PssAggClf':
            clf    =  PassiveAggressiveClassifier (random_state=0)
        elif name == 'Knn':
            clf    =  KNeighborsClassifier (random_state=0)
        elif name == 'RF':
            clf    =  RandomForestClassifier (random_state=0)
        elif name == 'NearCent':
            clf    =  NearestCentroid (random_state=0)
        elif name == 'MultNB':
            clf    =  MultinomialNB (random_state=0)
        elif name == 'BernNB':
            clf    =  BernoulliNB (random_state=0)    
        elif name == 'Svc':
            clf    =  SVC (probability=True, random_state=0)
        elif name == 'LSvc':
            clf    =  LinearSVC (random_state=0)
        elif name == 'Xgb':
            clf    =  xgb.XGBClassifier (random_state=0) # XGBRFClassifier()
        elif name == 'Catb' :                            # issues with CV
            clf    =  CatBoostClassifier (verbose=False, random_state=0)
        elif name == 'FCNN':
            clf    =  None    # init at  fit ()
        else:
            print('ERROR Best_clf_cv_transformer: invalid @param name \n')
            clf    = None
        self.isCV = True
        if 'isCV' in self.myparams:
            
            self.isCV = self.myparams['isCV']
        self.n_estimators = 1
        if 'n_estimators' in self.myparams:
            
            self.n_estimators = self.myparams['n_estimators']
        if 'params' in self.myparams:
            
            clf.set_params (**self.myparams['params'])
        if other_params:
            
            clf.set_params (**other_params)
        self.param_grid = None
        if 'param_grid' in self.myparams:
            
            self.param_grid = self.myparams['param_grid']
        self.clf = clf
        self.cv_score = 0
        self.name = name
        self._estimator_type='classifier'
        return    
    
    def fit (self, X, Y, **FIT_PARAMS):
        
        train_len = len (Y[pd.isnull(Y)==False])
        X, Y = X[:train_len], Y[:train_len]
        print ('training', self.name, 'for X.shape =', X.shape)
        n_jobs = -1    
        if self.name == 'FCNN':
            
            lrScheduler = LRScheduler (
                
                CyclicLR,
                base_lr=0.0001,
                max_lr=0.05,
                step_every='batch'
            )
            self.clf = NeuralNetClassifier (
                
                    module=MyModule,
                    module__inputCount=X.shape[1], 
                    module__outputCount=2, 
                    module__hiddenLayerCounts=[15],
                    max_epochs=1000,
                    # lr=0.01,
                    verbose=0,
                    # Shuffle training data on each epoch
                    iterator_train__shuffle=True,
                    callbacks=[('LRScheduler', lrScheduler), ('EarlyStopping', EarlyStopping (patience=20))]
            )
            X = X.astype (np.float32)
            Y = Y.astype (np.int64)
            n_jobs = None
        if self.isCV:
            if self.param_grid:

                gridSearchCV = GridSearchCV (
                    self.clf, self.param_grid, iid=False, cv=self.cv, scoring=CV_SCORERS, 
                    refit='f1_score', n_jobs=n_jobs
                )
                gridSearchCV.fit (X, Y)  
                print (self.name, ": Best_clf_cv_transformer: Best parameter (CV score=%0.3f):" % gridSearchCV.best_score_)
                print (gridSearchCV.best_params_)
                self.clf = gridSearchCV.best_estimator_
                self.cv_score = gridSearchCV.best_score_
                if self.name == 'LSvc':
                    
                    self.clf = CalibratedClassifierCV (self.clf)
                    # self.clf.fit (X, Y)
                # global RESULTS
                # RESULTS.append(benchmark(self.clf))
                # TODO: Plot scores for each split, and get its' variance
            else:

                if self.name == 'LSvc':
                    
                    self.clf = CalibratedClassifierCV (self.clf)
                print (self.name, ': Best_clf_cv_transformer: starting CV =', self.cv)
                if self.name not in {'fss'}:  # {'RF', 'Catb', 'FCNN'}:
                    
                    cv_results = cross_validate (self.clf, X, Y, cv=int(self.cv))
                else:
                    
                    voting_clf = VotingClassifier (estimators=[(self.name, self.clf)])
                    cv_results = cross_validate (voting_clf, X, Y, cv=self.cv)
                self.cv_score = np.mean (cv_results['test_score'])
                # self.clf.fit (X, Y)
                print (self.name, ": cv_score:   %0.3f" % self.cv_score)
        else:
            if self.name == 'LSvc':
                self.clf = CalibratedClassifierCV (self.clf)
            # self.clf.fit (X, Y)
        if self.n_estimators > 1:
            self.clf = BaggingClassifier (base_estimator=self.clf, n_estimators=self.n_estimators)
        self.clf.fit (X, Y)
        print ("Done Fitting", self.name)
        return self
    
    def get_cv_score (self):
        return self.cv_score
    
    def transform (self, X, Y=None, **FIT_PARAMS):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)
            if not Y is None:
                Y = Y.astype (np.int64)        
        return self.clf.transform(X, Y)
    
    def predict (self, X, **FIT_PARAMS):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)      
        return self.clf.predict(X)
    
    def predict_proba (self, X):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)      
        return self.clf.predict_proba (X)
    
    def predict_log_proba (self, X):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)
        return self.clf.predict_log_proba (X)
    
    def score (self, X, Y, **FIT_PARAMS):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)
        return self.clf.score (X, Y, **FIT_PARAMS)
    
    def decision_function (self, X, **FIT_PARAMS):        
        if self.name == 'FCNN':
            X = X.astype (np.float32)
        return self.clf.decision_function (X)
    
    def set_params (self, **params):        
        self.myparams = params['myparams']
        params.pop ('myparams')
        self.myinit (**params)
        return self
    
    def get_params (self, deep=True):
        params = {'myparams': self.myparams}
        return params
    
    def apply (self, X):
        return self.clf.apply(X)
    
    def decision_path (self, X):
        return self.clf.decision_path (X)
    
    def staged_decision_function (self, X):
        return self.clf.staged_decision_function (X)
    
    def staged_predict (self, X):
        return self.clf.staged_predict (X)
    
    def staged_predict_proba (self, X):
        return self.clf.staged_predict_proba (X)
    
    def staged_score (self, X):
        return self.clf.staged_score (X)

In [7]:
def seed_all ():
    
    random.seed (SEED)
    np.random.seed (SEED)
    random.seed (SEED)
    
seed_all ()

# Read and preprocess the Data

In [8]:
def load_preprocess_data (filename='../input/jane-street-market-prediction/train.csv', isTrainData=True):
    
    dtype = None
    if isTrainData:
        
        dtype = {
            'date'      : 'int64', 
            'weight'    : 'float64',
            'resp'      : 'float64',
            'ts_id'     : 'int64',  
            'feature_0' : 'float64'
        }
    else:
        
        dtype = {
            'date'      : 'int64', 
            'weight'    : 'float64',
            'feature_0' : 'float64'
        }
    for i in range (1, 130):
        k = 'feature_' + str (i)
        dtype[k] = 'float32'
    
    X = pd.read_csv (filename, dtype=dtype)
    
    resp_cols   = ['resp_1', 'resp_2', 'resp_3','resp_4', 'resp']    
    X           = X.query ('date > 85')
    X           = X[X['weight'] != 0].reset_index (drop = True)
    y           = np.stack ([(X[c] > 0).astype ('int') for c in resp_cols]).T
    f_columns   = [c for c in X.columns if "feature" in c]    
    Weights     = X['weight'].values.reshape ((-1,1))
    
    if isTrainData:
        X.drop (columns=['date', 'weight', 'resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'ts_id'], inplace=True)
    else:
        X.drop (columns=['date', 'weight'], inplace=True)
          
    preprocess_pipe =  Pipeline ([
        ("imputer", SimpleImputer (missing_values=np.nan, strategy='mean')),
        # ("stand",   StandardScaler (with_mean=False))
    ])
    X = preprocess_pipe.fit_transform (X)
    
    X = np.hstack ((X, Weights))
    
    X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.05)
    del X, y
    gc.collect ()
    
    W_train = X_train[:, -1]
    X_train = X_train[:, :-1]
    W_test  = X_test[:, -1]
    X_test  = X_test[:, :-1]
    return X_train, X_test, y_train, y_test, W_train, W_test, preprocess_pipe

In [9]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST, W_train, W_test, preprocess_pipe = load_preprocess_data ()
gc.collect ()
X_TRAIN.shape,  Y_TRAIN.shape

((1492844, 130), (1492844, 5))

In [10]:
# TODO: comment this
# X_TRAIN = X_TRAIN[:50000]
# Y_TRAIN = Y_TRAIN[:50000]

In [11]:
def learning_rate_010_decay_power_09 (current_iter):
    
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power (.995, current_iter)
    return lr if lr > 1e-2 else 1e-2


In [12]:
# n_estimators is set to a "large value" say 5000. The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
# or
# train till a small say 600 rounds only and select the best esimator, then trail this estimator to 10,000 rounds with early stopping

# Global Var
FIT_PARAMS= {
    "early_stopping_rounds":30, 
    "eval_metric" : 'auc', 
    "eval_set" : [(X_TEST, Y_TEST[:,-1])],
    'eval_names': ['valid'],
    'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_09)],
    'verbose': 50,
    'categorical_feature': 'auto'
}

# Search for the best model under constraints

param_test = {
     'num_leaves': sp_randint(6, 50), 
     'min_child_samples': sp_randint(100, 500), 
     'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
     'subsample': sp_uniform (loc=0.2, scale=0.8), 
     'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
     'reg_alpha' : [0, 1e-1, 1, 5,  10],
     'reg_lambda': [0, 1e-1, 1, 10, 20]
}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 4

# do a random search
lgb_clf = lgb.LGBMClassifier (max_depth=-1, random_state=SEED, silent=True, metric='None', n_jobs=8, n_estimators=N_ESTIMATORS)

gs = RandomizedSearchCV (
    estimator=lgb_clf, 
    param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=False
)

# uncomment this to train from scratch

gs.fit (X_TRAIN, Y_TRAIN, **FIT_PARAMS)
print ('Best score reached: {} with params: {} '.format (gs.best_score_, gs.best_params_))

# Opt Params

In [13]:
# [600]	valid's auc: 0.601937
# Best score reached: 0.598930845392645 with params: {'colsample_bytree': 0.6681799297021748, 'min_child_samples': 191, 'min_child_weight': 1, 'num_leaves': 46, 'reg_alpha': 0, 'reg_lambda': 0, 'subsample': 0.8720649952803985}

In [14]:
# Global Var
OPT_PARAMS_1 = {'n_estimators': N_ESTIMATORS, 'colsample_bytree': 0.668, 'min_child_samples': 150, 'min_child_weight': 1, 'num_leaves': 90, 'reg_alpha': 0, 'reg_lambda': 0.001, 'subsample': 0.87}
OPT_PARAMS_2 = {'n_estimators': N_ESTIMATORS, 'colsample_bytree': 0.668, 'min_child_samples': 190, 'min_child_weight': 1, 'num_leaves': 100, 'reg_alpha': 0, 'reg_lambda': 0.001, 'subsample': 0.87}

# Create clf from the best params

In [15]:
def create_train_lgbm (X_train, y_train, component):
    
    if component == 1:
        opt_params = deepcopy (OPT_PARAMS_1)
    else:
        opt_params = deepcopy (OPT_PARAMS_2)
        
    # find #best-iterations
    lgb_clf_1 = lgb.LGBMClassifier (**opt_params)   #;print ('X_train.shape =', X_train.shape),  ;print ('y_train.shape =', y_train.shape)
    lgb_clf_1.fit (X_train, y_train, **FIT_PARAMS)

    # refit on the #best-iterations
    if lgb_clf_1.best_iteration_ != N_ESTIMATORS:
        opt_params['n_estimators'] = lgb_clf_1.best_iteration_
        lgb_clf_1  = lgb.LGBMClassifier (**opt_params)
        lgb_clf_1.fit (X_train, y_train, **FIT_PARAMS)
        
    # save
    # pickleSave (lgb_clf_1, str (model_id)+'_'+str (component)+'_lgbm.bin')

    # sanity test
    X_temp = X_train[10].reshape ((1, -1))
    np.round (lgb_clf_1.predict (X_temp)).astype (int)
    return lgb_clf_1

In [16]:
def getLgbs ():
    
    LGBS    = []
    for model_id in range (5):
        
        y_train = Y_TRAIN[:, model_id]        #;print ('X_TRAIN.shape =', X_TRAIN.shape),  ;print ('y_train.shape =', y_train.shape)
        lgbm_1  = create_train_lgbm (X_TRAIN, y_train, 1)
        lgbm_2  = create_train_lgbm (X_TRAIN, y_train, 2)
        LGBS.append ((lgbm_1, lgbm_2))

    # save
    pickleSave (LGBS, 'lgbs.bin')
    return LGBS

# Stacking clf

In [17]:
def getSclfs ():
    
    SCLFS = []
    for model_id in range (5):

        sclf = StackingClassifier (classifiers=LGBS[model_id], fit_base_estimators=False, 
                                   use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.01, 0.05, 0.1, 1]} }) )
        sclf.name = 'sclf_' + str (model_id)
        y_train   = Y_TRAIN[:, model_id]
        sclf.fit (X_TRAIN, y_train)
        SCLFS.append (sclf)

    # save
    pickleSave (SCLFS, 'sclfs.bin')

    # sanity test
    X_temp = X_TRAIN[10].reshape ((1, -1))
    np.round (SCLFS[0].predict (X_temp)).astype (int)
    return SCLFS

In [18]:
# Global Vars, Calling training
LGBS  = getLgbs ()
SCLFS = getSclfs ()

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[8]	valid's auc: 0.522514	valid's binary_logloss: 0.692237
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[8]	valid's auc: 0.522514	valid's binary_logloss: 0.692237
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[7]	valid's auc: 0.52133	valid's binary_logloss: 0.692269
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[7]	valid's auc: 0.52133	valid's binary_logloss: 0.692269
Training until validation scores don't improve for 30 rounds
[50]	valid's auc: 0.536762	valid's binary_logloss: 0.691212
[100]	valid's auc: 0.543966	valid's binary_logloss: 0.690088
[150]	valid's auc: 0.547389	valid's binary_logloss: 0.689355
[200]	valid's auc: 0.549138	valid's binary_logloss: 0.688918
[250]	valid's auc: 0.550247	valid's binary_logloss: 

In [19]:
# use pre trained models
# LGBS  = unpickle ('../input/janelgbmstackedlsvc-1/lgbs.bin')
# SCLFS = unpickle ('../input/janelgbmstackedlsvc-1/sclfs.bin')

# Predict for X_test

THRESH = 0.50

print ('Finding the optimal probability threshold..')

Y_TRAIN = np.median (Y_TRAIN, axis=1)

# Random init threshold for gradient descent
TH = torch.tensor (0.50, device=DEVICE, dtype=float, requires_grad=True)

# Loss (W*Actual, W * Ceil (pr-TH))
W_train = torch.tensor (W_train).float ()                   #;print ('W_train.shape =', W_train.size ())
Y_TRAIN = torch.tensor (Y_TRAIN).float ()
X_TRAIN = torch.tensor (X_TRAIN).float ()

A  = torch.tensor (W_train * Y_TRAIN)                       #;print ('A.shape =', A.size ())

# create batches to fit in memory
epochs  = 10
lr      = 0.1
batches = 2

for e in range (epochs):
    
    lr *= 0.9
    lr  = 0.01 if lr < 0.01 else lr
    for b in range (batches):

        batch_siz = X_TRAIN.size(0)//10
        b_A       = A[b*batch_siz : (b+1)*batch_siz]
        b_W_train = W_train[b*batch_siz : (b+1)*batch_siz]
        b_X_train = X_TRAIN[b*batch_siz : (b+1)*batch_siz]
        b_pr      = torch.tensor (predict (b_X_train.numpy ()))        #;print ('pr.shape =', b_pr.size ())

        b_P       = b_W_train * torch.ceil (b_pr - TH) 
        # print ('b_W_train =', b_W_train, 'b_pr =', b_pr, 'torch.ceil(b_pr - TH) =', torch.ceil(b_pr - TH), 'b_P =', b_P)
        # print ('(b_pr - TH).shape =', (b_pr - TH).size (), 'b_W_train.shape =', b_W_train.shape, 'b_P.shape =', b_P.shape)
        b_loss    = (b_A - b_P).pow (2).mean ()
        b_loss.backward ()    
        with torch.no_grad ():
            TH -= lr * TH.grad
        TH.grad = None    

        if b % batches == batches-1:
            print('loss =', b_loss.item ())

THRESH = TH.detach ().numpy ()
print ('THRESH =', THRESH)
THRESH

# Prediction

In [20]:
def predict (test_df, isRetProb=False):
    
    test_df.drop (columns=['weight', 'date'], inplace=True)
    test_df.reset_index (drop=True, inplace=True)
    X_test = preprocess_pipe.transform (test_df).reshape ((-1, 130))
    
    y_probs = []
    for sclf in SCLFS:
        y_p = sclf.predict_proba (X_test).reshape ((-1, 2))[:, 1].reshape ((-1, 1))
        y_probs.append (y_p)
        
    y_probs = np.hstack (y_probs)                 #;print ('y_probs.shape =', y_probs.shape, 'X_test.shape =', X_test.shape)
    pred_pr = np.median (y_probs, axis=1)         #;print ('pred_pr.shape =', pred_pr.shape, 'pred_pr =', pred_pr) 
    y_pred  = (pred_pr >= 0.5).astype (int)
    if isRetProb:
        return y_pred, pred_pr
    else:
        return y_pred

# Test Pred

In [21]:
filename='../input/jane-street-market-prediction/train.csv'
# eval the prediction
dtype = {
    'date'      : 'int64', 
    'weight'    : 'float64',
    'resp'      : 'float64',
    'ts_id'     : 'int64',  
    'feature_0' : 'float64'
}
for i in range (1, 130):
    k = 'feature_' + str (i)
    dtype[k] = 'float32'

test_df   = pd.read_csv (filename, dtype=dtype)
resp_cols = ['resp_1', 'resp_2', 'resp_3','resp_4', 'resp']   
Y_test_df = np.stack ([(test_df[c] > 0).astype ('int') for c in resp_cols]).T
test_df.drop (columns=resp_cols+['ts_id'], inplace=True) 
Y_test_df = np.mean (Y_test_df, axis=1)
pred_pr   = predict (test_df, True)[1]
pred_pr.shape

(2390491,)

# Submission

In [22]:
import janestreet
env      = janestreet.make_env () # initialize the environment
env_iter = env.iter_test ()       # an iterator which loops over the test set

# For direct submission, without using the Trainer class

In [23]:
for test_df, pred_df in env_iter:
    if test_df["weight"].item () > 0:
        
        predictions    = predict (test_df)
        pred_df.action = predictions
    else:
        pred_df.action = 0
        
    # print (pred_df)
    # print ("--------------")
    env.predict (pred_df)

In [24]:
print ('Done !')

Done !
